In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - 稳定扩散V1.5（LoRA微调）

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_finetuning_lora.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_finetuning_lora.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_finetuning_lora.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
    （建议使用Python-3 CPU笔记本）
  </td>
</table>

## 概览

这本笔记本演示了如何使用[LoRA](https://huggingface.co/docs/diffusers/training/text2image#lora)对runwayml/stable-diffusion-v1-5进行微调，并在Vertex AI上部署以进行在线预测。

### 目标

- 使用[LoRA](https://huggingface.co/docs/diffusers/training/text2image#lora)微调stable-diffusion-v1.5模型。

### 成本

此教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

在您开始之前

**注意**：Jupyter会将以`!`前缀开头的行视为shell命令，并将以`$`前缀开头的Python变量插入这些命令中。

### 设置笔记本

仅限Colab
如果您正在使用Workbench，请运行以下命令并跳过此部分。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    ! pip3 install --upgrade pip
    # Remove wrong cublas version.
    ! pip3 uninstall nvidia_cublas_cu11 --yes

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### 仅工作台
1. 点击[此链接](https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_deployment_1_5.ipynb)将笔记本部署到Vertex AI工作台实例中。
2. 选择“创建新笔记本”。
3. 点击“高级选项”。
4. 在**环境**选项卡中，选择**操作系统**为“Debian 10”，选择**环境**为“自定义容器”。
5. 将`Docker容器镜像`设置为`us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/transformers-notebook`。
6. 在**机器配置**下，选择1个`T4` GPU，并选择“自动为我安装NVIDIA GPU驱动程序”。
7. 点击“创建”以创建Vertex AI工作台实例。

### 设置Google Cloud项目

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您将获得$300的免费信用，用于支付计算/存储成本。

2. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个Cloud Storage存储桶](https://cloud.google.com/storage/docs/creating-buckets)用于存储实验输出。

5. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)，并为其分配`Vertex AI User`和`Storage Object Admin`角色，用于部署微调模型到Vertex AI端点。

填写以下实验环境的变量：

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

初始化 Vertex-AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

定义常量

In [ ]:
# The pre-built training docker images. They contain training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20231130_0936_RC00"

# The pre-built serving docker images. They contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240605_1400_RC00"

### 定义常用函数

In [ ]:
import base64
import os
from datetime import datetime
from io import BytesIO

import requests
from google.cloud import aiplatform
from PIL import Image


def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(
    model_name,
    base_model_id,
    lora_id,
    task,
    machine_type="g2-standard-8",
    accelerator_type="NVIDIA_L4",
):
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": base_model_id,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    if lora_id:
        serving_env["LORA_ID"] = lora_id
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint

## 使用LoRA进行微调和部署

### 微调

本节使用[LoRA](https://arxiv.org/abs/2106.09685)对[stable-diffusion-v1.5](https://huggingface.co/runwayml/stable-diffusion-v1-5)模型进行微调，使用[lambdalabs/pokemon-blip-captions](https://huggingface.co/datasets/lambdalabs/pokemon-blip-captions)。

使用LoRA进行微调

在微调作业完成后，LoRA权重将被保存，可通过[StableDiffusionPipeline](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/text2img)加载以进行推理。

In [ ]:
# The pre-trained model to be loaded.
model_id = "runwayml/stable-diffusion-v1-5"
DATASET_NAME = "lambdalabs/pokemon-blip-captions"

# Output path.
output_dir_lora = f"/gcs/{GCS_BUCKET}/lora/output"
gs_output_dir_lora = f"gs://{GCS_BUCKET}/lora/output"

# Worker pool spec.
machine_type = "n1-standard-8"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 1


# Setup training job.
job_name = create_job_name("lora-stable-diffusion")
job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)

# Set task to "text-to-image-lora" to finetune with LoRA.
# Pass training arguments and launch job.
# See https://github.com/huggingface/diffusers/blob/87ae330056f6942817656c8f7146283e90cf986b/examples/text_to_image/train_text_to_image_lora.py#L84
# for a full list of training arguments.
model = job.run(
    args=[
        "--task=text-to-image-lora",
        f"--pretrained_model_name_or_path={model_id}",
        f"--dataset_name={DATASET_NAME}",
        "--caption_column=text",
        "--resolution=512",
        "--random_flip",
        "--train_batch_size=1",
        "--num_train_epochs=20",
        "--checkpointing_steps=5000",
        "--learning_rate=1e-01",
        "--lr_scheduler=constant",
        "--lr_warmup_steps=0",
        f"--output_dir={output_dir_lora}",
        "--validation_prompt='cute dragon creature'",
        "--seed=42",
    ],
    replica_count=num_nodes,
    machine_type=machine_type,
    accelerator_type=gpu_type,
    accelerator_count=num_gpus,
)

### 部署

为文本到图像任务部署稳定的扩散模型。

一旦部署完成，您可以向端点发送一批文本提示以生成图像。

In [ ]:
model, endpoint = deploy_model(
    model_name=create_job_name(prefix="lora-stable-diffusion"),
    model_id=model_id,
    lora_id=gs_output_dir_lora,
    task="text-to-image",
)

请注意：模型权重将在部署成功后进行下载。因此，在上述模型部署步骤成功之后，需要额外等待5分钟，然后再运行下面的下一个步骤。否则，当您发送请求到端点时，可能会看到“ServiceUnavailable: 503 502:Bad Gateway”错误。

In [ ]:
instances = [
    {"prompt": "A pokemon with green eyes and red legs."},
    {"prompt": "A pokemon with yellow tails and blue arms."},
    {"prompt": "A pokemon with three heads and green feet."},
    {"prompt": "A dragon pokemon with red and white stripes."},
]
response = endpoint.predict(instances=instances)
images = [base64_to_image(image) for image in response.predictions]
image_grid(images)

整理。

In [ ]:
# Undeploy model and delete endpoints.
endpoint.delete(force=True)
# Delete models.
model.delete()